# 1번

## 1-1번 불균형 데이터 파악
- TARGET열의 IR을 파악한 후, over/under sampling의 필요 여부를 판단하세요.
    - TARGET열의 레이블 값 별 데이터 개수를 출력하고, 불만족의 비율을 구하시오.
    - IR>10 일 경우 over/under sampling이 필요하다고 가정.
        - target==0: 만족, target==1: 불만족 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

df=pd.read_csv('./train.csv', encoding='latin-1')
print(df.shape)
df.head()

## 1-2번 데이터 전처리
    - var3 열의 이상점을 50%값으로 교체하세요.
    - 머신러닝 모델에 영향을 주지 않을 것이라 판단되는 하나의 열을 제거하세요.
    - 데이터를 학습 데이터와 테스트 데이터로 분리하세요.
        - test_size=0.2
        - target열을 y값으로
    - 학습 데이터와 테스트 데이터의 TARGET열의 레이블 값 비율을 구하세요

In [ ]:
from sklearn.model_selection import train_test_split

## 1-3번 OverSampling을 통해 평가지표 구하기
    - XGBoost를 이용하세요.
        - n_estimators=1000,random_state=156,learning_rate=0.02, max_depth=7, min_child_weight=1, colsample_bytree=0.75, reg_alpha=0.03
    - smote를 적용 하기 전의 성능과 smote 적용 후의 성능을 비교하세요.
    (주의: 모델을 돌리는데 꽤 오랜 시간이 걸립니다ㅜ)
    - 재현율을 비교하며 over_sampling에 대한 본인의 의견을 말씀해주세요.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, precision_recall_curve, roc_curve

In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion=confusion_matrix(y_test, pred)
    accuracy=accuracy_score(y_test, pred)
    precision=precision_score(y_test, pred)
    recall=recall_score(y_test, pred)
    f1=f1_score(y_test, pred)
    roc_auc=roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}, AUC: {4:.4f}'
          .format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    model.fit(ftr_train, tgt_train)
    pred=model.predict(ftr_test)
    pred_proba=model.predict_proba(ftr_test)[:, 1]
    get_clf_eval(tgt_test, pred, pred_proba)

In [ ]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

## 1-4번 기본 스태킹 모델

※ SMOTE 적용하기 전의 데이터를 이용해주세요  

1) 개별 모델을 생성,학습하고 각 모델의 예측 정확도를 구하시오.
- 개별 모델 : KNN, 랜덤 포레스트, 결정 트리, 에이다부스트 
- 최종 모델 : 로지스틱 회귀

< 각 모델의 파라미터 >  
KNN : n_neighbors=4   
랜덤 포레스트 : n_estimators=100, random_state=0   
에이다부스트 : n_estimators=100   
로지스틱 회귀 : C=10

In [ ]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

2) 최종 메타 모델의 예측 정확도를 구하시오.

## 1-5번 CV 세트 기반의 스태킹

1) 모델별로 get_stacking_base_datasets() 함수를 호출해 각각 메타 모델이 추후에 사용할 학습용, 테스트용 데이터 세트를 반환하시오.  
   (주의) 실행하는데 시간이 오래걸릴 수 있습니다
   - n_folds=3   

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, random_state=0)
    #kf=StratifiedKFold(n_splits=n_folds)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n.iloc[train_index] 
        y_tr = y_train_n.iloc[train_index] 
        X_te = X_train_n.iloc[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

2) get_stacking_base_datasets() 호출로 반환된 각 모델별 학습 데이터와 테스트 데이터를 합치고 최종 메타 모델의 예측 정확도를 구하시오.

# 2번

1) mushroom 데이터를 불러와서 결측값를 유무 확인해주세요

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
df = pd.read_csv("C:/python/mushrooms.csv")

2) describe를 통해 각 열의 특징을 나타내세요

3) 'class'열의 고유항목 수를 나타낸 뒤, 'class'열 고유값의 count를 barplot로 나타내보세요
 - seaborn을 이용해 barplot을 나타내세요
 - xlabel: 'class' , ylabel: 'count'
 - title: 'Number of poisonous/edible mushrooms'

In [ ]:
#LabelEncoder를 사용하기 위해 유형을 'category'로 변경후 LabelEncoder적용
df = df.astype('category')
df.dtypes

labelencoder=LabelEncoder()
for column in df.columns:
    df[column] = labelencoder.fit_transform(df[column])
df.head()

4) veil-type" 열 은 0 이고 데이터에 기여하지 않으므로 제거하세요

## 분류모델 적용

5) 데이터 셋을 80%는 train 데이터, 20%는 test 데이터로 분리해주세요. (타겟은 class 열입니다)

6) 의사결정트리와 랜덤포레스트 분류기를 만들고 학습시킨 뒤, classification_report를 나타내보세요
- 설정은 디폴트 값 사용

# 3번

## 오토ML

In [ ]:
from pycaret.classification import *
import pandas as pd
data = pd.read_csv('mushrooms.csv')
data.head()

1) setup() 메서드를 사용하여 AutoML을 위한 초기 셋팅을 완료하세요
- target = 'class'
- train_size=0.8 설정

2) compare_models() 메서드를 사용하여 성능 상위 3개 모델을 변수에 저장해주세요
- 교차검증 폴드 수는 2로 지정

3) (2)번에서 만든 모델을 활용하여 앙상블 모델을 만들어주세요
- 교차검증 폴드 수는 2로 지정
- 하드보팅 사용

4) (3)번에서 만든 앙상블 모델로 hold-out 세트에 예측해주세요